# Legacy User Analytics Pipeline

This notebook demonstrates a **legacy** approach to analyzing user analytics data using pandas. 

⚠️ **Warning**: This code contains several performance bottlenecks and inefficient patterns that should be optimized for production use.

## Dataset Overview
- **User Data**: Basic user information and demographics
- **Session Data**: User session information with device and location data
- **Event Data**: Individual user events and interactions

Let's start by loading and exploring our data...


In [1]:
%pip install -qq pandas numpy

Note: you may need to restart the kernel to use updated packages.


import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

print("Loading user analytics data...")
start_time = time.time()

# Load our datasets
users_df = pd.read_csv('user_data.csv')
sessions_df = pd.read_csv('user_sessions.csv')
events_df = pd.read_csv('user_events.csv')

# Convert datetime columns
users_df['signup_date'] = pd.to_datetime(users_df['signup_date'])
users_df['first_seen'] = pd.to_datetime(users_df['first_seen'])
users_df['last_seen'] = pd.to_datetime(users_df['last_seen'])

sessions_df['session_start'] = pd.to_datetime(sessions_df['session_start'])
events_df['event_time'] = pd.to_datetime(events_df['event_time'])

load_time = time.time() - start_time
print(f"Data loaded in {load_time:.2f} seconds")
print(f"Users: {len(users_df):,}")
print(f"Sessions: {len(sessions_df):,}")
print(f"Events: {len(events_df):,}")


## Anti-Pattern #1: Using iterrows() for Row-by-Row Processing

Let's calculate user engagement scores using the inefficient `iterrows()` method instead of vectorized operations.


In [3]:
def calculate_engagement_score_slow(users_df):
    """
    INEFFICIENT: Using iterrows() to calculate engagement scores
    This is much slower than vectorized operations
    """
    print("Calculating engagement scores (inefficient method)...")
    start_time = time.time()
    
    engagement_scores = []
    
    # BAD: Using iterrows() - very slow for large datasets
    for index, row in users_df.iterrows():
        # Calculate days since signup
        days_since_signup = (row['last_seen'] - row['signup_date']).days
        
        # Calculate engagement score based on sessions and revenue
        if days_since_signup > 0:
            session_rate = row['total_sessions'] / days_since_signup
            revenue_per_session = row['total_revenue'] / row['total_sessions'] if row['total_sessions'] > 0 else 0
            engagement_score = (session_rate * 0.6) + (revenue_per_session * 0.4)
        else:
            engagement_score = 0
        
        engagement_scores.append(engagement_score)
    
    users_df['engagement_score'] = engagement_scores
    
    elapsed_time = time.time() - start_time
    print(f"Engagement scores calculated in {elapsed_time:.2f} seconds")
    return users_df

# Apply the slow function
users_df = calculate_engagement_score_slow(users_df)
print(f"Average engagement score: {users_df['engagement_score'].mean():.4f}")


Calculating engagement scores (inefficient method)...
Engagement scores calculated in 0.02 seconds
Average engagement score: 30.9843


## Anti-Pattern #2: Inefficient String Operations with apply()

Let's categorize email domains using inefficient string operations and unnecessary apply() calls.


In [4]:
def categorize_email_domain_slow(email_domain):
    """
    INEFFICIENT: Using multiple string operations in apply()
    This should be replaced with vectorized string operations
    """
    # BAD: Inefficient string operations inside apply()
    email_domain = str(email_domain).lower().strip()
    
    # Multiple if-else statements that could be optimized
    if email_domain.find('gmail') != -1:
        return 'Personal'
    elif email_domain.find('yahoo') != -1:
        return 'Personal'
    elif email_domain.find('hotmail') != -1:
        return 'Personal'
    elif email_domain.find('outlook') != -1:
        return 'Personal'
    elif email_domain.find('company') != -1:
        return 'Corporate'
    else:
        return 'Other'

print("Categorizing email domains (inefficient method)...")
start_time = time.time()

# BAD: Using apply() with complex lambda when vectorized operations would be faster
users_df['email_category'] = users_df['email_domain'].apply(
    lambda x: categorize_email_domain_slow(x)
)

# More inefficient string operations
users_df['email_provider'] = users_df['email_domain'].apply(
    lambda x: str(x).split('.')[0].upper() if '.' in str(x) else 'UNKNOWN'
)

elapsed_time = time.time() - start_time
print(f"Email categorization completed in {elapsed_time:.2f} seconds")

# Show the results
print("\nEmail category distribution:")
print(users_df['email_category'].value_counts())


Categorizing email domains (inefficient method)...
Email categorization completed in 0.00 seconds

Email category distribution:
email_category
Personal     800
Corporate    193
Name: count, dtype: int64


## Anti-Pattern #3: Nested Loops for Aggregations

Let's calculate user behavior patterns using nested loops instead of efficient groupby operations.


In [5]:
def calculate_user_metrics_slow(users_df, sessions_df, events_df):
    """
    INEFFICIENT: Using nested loops for aggregations
    This should be replaced with groupby operations
    """
    print("Calculating user metrics (inefficient nested loops)...")
    start_time = time.time()
    
    user_metrics = []
    
    # BAD: Nested loops for aggregations - very slow!
    for user_id in users_df['user_id'].unique():
        user_sessions = []
        user_events = []
        
        # Inner loop 1: Find all sessions for this user
        for _, session in sessions_df.iterrows():
            if session['user_id'] == user_id:
                user_sessions.append(session)
        
        # Inner loop 2: Find all events for this user
        for _, event in events_df.iterrows():
            if event['user_id'] == user_id:
                user_events.append(event)
        
        # Calculate metrics manually instead of using built-in functions
        total_session_duration = 0
        for session in user_sessions:
            total_session_duration += session['session_duration']
        
        avg_session_duration = total_session_duration / len(user_sessions) if user_sessions else 0
        
        # Count events by type manually
        event_counts = {}
        for event in user_events:
            event_type = event['event_type']
            if event_type not in event_counts:
                event_counts[event_type] = 0
            event_counts[event_type] += 1
        
        # Calculate conversion rate manually
        purchases = event_counts.get('purchase', 0)
        total_events = len(user_events)
        conversion_rate = purchases / total_events if total_events > 0 else 0
        
        user_metrics.append({
            'user_id': user_id,
            'avg_session_duration': avg_session_duration,
            'total_events': total_events,
            'purchases': purchases,
            'conversion_rate': conversion_rate
        })
    
    elapsed_time = time.time() - start_time
    print(f"User metrics calculated in {elapsed_time:.2f} seconds")
    
    return pd.DataFrame(user_metrics)

# Apply the slow function
user_metrics_df = calculate_user_metrics_slow(users_df, sessions_df, events_df)
print(f"Calculated metrics for {len(user_metrics_df)} users")
print(f"Average conversion rate: {user_metrics_df['conversion_rate'].mean():.4f}")


Calculating user metrics (inefficient nested loops)...
User metrics calculated in 274.26 seconds
Calculated metrics for 993 users
Average conversion rate: 0.1211


## Anti-Pattern #4: Memory-Inefficient Operations

Let's create multiple unnecessary intermediate DataFrames and perform inefficient merges.


In [9]:
def create_user_summary_slow(users_df, sessions_df, events_df):
    """
    INEFFICIENT: Creating multiple unnecessary intermediate DataFrames
    and performing inefficient merges
    """
    print("Creating user summary (memory-inefficient method)...")
    start_time = time.time()
    
    # BAD: Creating many intermediate DataFrames instead of chaining operations
    temp_df1 = users_df.copy()
    temp_df2 = temp_df1.copy()
    temp_df3 = temp_df2.copy()
    
    # BAD: Multiple separate operations instead of efficient chaining
    sessions_by_user = sessions_df.groupby('user_id').agg({
        'session_duration': ['mean', 'sum', 'count'],
        'pages_visited': ['mean', 'sum'],
        'revenue': ['sum', 'mean']
    }).reset_index()
    
    # BAD: Flattening column names inefficiently
    sessions_by_user.columns = ['user_id', 'avg_session_duration', 'total_session_duration', 
                               'session_count', 'avg_pages_visited', 'total_pages_visited',
                               'total_revenue', 'avg_revenue_per_session']
    
    # BAD: Creating another intermediate DataFrame
    events_by_user = events_df.groupby('user_id').agg({
        'event_id': 'count',
        'value': ['sum', 'mean']
    }).reset_index()
    
    events_by_user.columns = ['user_id', 'total_events', 'total_event_value', 'avg_event_value']
    
    # BAD: Inefficient merges - creating copies each time
    result_df = temp_df3.copy()
    result_df = result_df.merge(sessions_by_user, on='user_id', how='left')
    result_df = result_df.merge(events_by_user, on='user_id', how='left')
    
    # Fill only columns that exist in result_df to avoid KeyError
    numeric_columns = [
        'avg_session_duration', 'total_session_duration', 'session_count',
        'avg_pages_visited', 'total_pages_visited', 'total_revenue',
        'avg_revenue_per_session', 'total_events', 'total_event_value',
        'avg_event_value'
    ]
    for col in numeric_columns:
        if col in result_df.columns:
            result_df[col] = result_df[col].fillna(0)

    final_df = result_df.copy()

    # Only create 'user_tier' if 'total_revenue' exists to avoid KeyError
    if 'total_revenue' in final_df.columns:
        final_df['user_tier'] = final_df['total_revenue'].apply(
            lambda x: 'Premium' if x > 100 else 'Standard' if x > 50 else 'Basic'
        )
    else:
        final_df['user_tier'] = 'Basic'

    elapsed_time = time.time() - start_time
    print(f"User summary created in {elapsed_time:.2f} seconds")

    return final_df

# Apply the memory-inefficient function
user_summary_df = create_user_summary_slow(users_df, sessions_df, events_df)
print(f"Created summary for {len(user_summary_df)} users")
print(f"Memory usage: {user_summary_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Show user tier distribution
print("\nUser tier distribution:")
print(user_summary_df['user_tier'].value_counts())


Creating user summary (memory-inefficient method)...
User summary created in 0.01 seconds
Created summary for 993 users
Memory usage: 0.45 MB

User tier distribution:
user_tier
Basic    993
Name: count, dtype: int64


## Final Results and Summary

Let's display a sample of our final results and summarize the performance issues in this pipeline.


In [12]:
# Display sample results
print("Sample of final user summary:")
print(user_summary_df[['user_id', 'email_category', 'engagement_score', 'user_tier', 'session_count', 'total_events']].head(10))

print("\n💡 These patterns can be optimized using modern pandas techniques!")
print("💡 Expected performance improvement: 5-10x faster execution")
print("💡 Expected memory reduction: 50-70% less memory usage")


Sample of final user summary:
     user_id email_category  engagement_score user_tier  session_count  \
0  user_0001       Personal         33.044702     Basic              6   
1  user_0002       Personal         20.100703     Basic              6   
2  user_0003       Personal          0.060000     Basic              1   
3  user_0004      Corporate          0.013333     Basic              5   
4  user_0005       Personal          0.040000     Basic              1   
5  user_0006       Personal         56.976155     Basic              6   
6  user_0007       Personal         27.012802     Basic              7   
7  user_0008       Personal         39.366951     Basic              3   
8  user_0009      Corporate          2.661873     Basic              8   
9  user_0010       Personal        100.459747     Basic              4   

   total_events  
0          34.0  
1          37.0  
2           0.0  
3          25.0  
4           5.0  
5          30.0  
6          39.0  
7          